In [3]:
import pandas as pd

file_path = "data - sample.xlsx"
xls = pd.ExcelFile("C:/Users/shubh/Downloads/Data Engineering/Data Engineering/data - sample.xlsx")

attendance_records = xls.parse('Attendance_data')
student_info = xls.parse('Student_data')

attendance_records['attendance_date'] = pd.to_datetime(attendance_records['attendance_date'])
attendance_records = attendance_records.sort_values(by=['student_id', 'attendance_date'])

def detect_long_absences(data, threshold=3):
    absences = []
    for sid, records in data.groupby('student_id'):
        records = records.reset_index(drop=True)
        absent_days = 0
        period_start = None

        for idx, row in records.iterrows():
            if row['status'] == 'Absent':
                absent_days += 1
                if absent_days == 1:
                    period_start = row['attendance_date']
                if absent_days > threshold:
                    period_end = row['attendance_date']
            else:
                if absent_days > threshold:
                    period_end = records.loc[idx - 1, 'attendance_date']
                    absences.append((sid, period_start, period_end, absent_days))
                absent_days = 0

        if absent_days > threshold:
            period_end = records.iloc[-1]['attendance_date']
            absences.append((sid, period_start, period_end, absent_days))
    
    return absences

absence_data = detect_long_absences(attendance_records)

absence_df = pd.DataFrame(absence_data, columns=[
    'student_id', 'absence_start_date', 'absence_end_date', 'total_absent_days'
])

print(absence_df)


   student_id absence_start_date absence_end_date  total_absent_days
0         107         2024-03-22       2024-03-25                  4
1         108         2024-03-21       2024-11-05                  5


In [5]:
import pandas as pd
import re

students = pd.DataFrame({
    'student_id': [101, 102, 103, 104, 105],
    'student_name': ['Alice Johnson', 'Bob Smith', 'Charlie Brown', 'David Lee', 'Eva White'],
    'parent_email': [
        'alice_parent@example.com',
        'bob_parent@example.com',
        'invalid_email.com',
        'invalid_email.com',
        'eva_white@example.com'
    ]
})

attendance = pd.DataFrame({
    'student_id': [101]*6 + [102]*6 + [103]*6 + [104]*6 + [105]*6,
    'date': pd.date_range(start='2024-03-01', periods=6).tolist() * 5,
    'status': ['Present', 'Absent', 'Absent', 'Absent', 'Absent', 'Present'] * 5
})

attendance['date'] = pd.to_datetime(attendance['date'])
attendance = attendance.sort_values(by=['student_id', 'date'])

def find_absent_streaks(data, limit=3):
    records = []
    for sid, group in data.groupby('student_id'):
        group = group.reset_index(drop=True)
        streak = 0
        start = None
        for i, row in group.iterrows():
            if row['status'] == 'Absent':
                streak += 1
                if streak == 1:
                    start = row['date']
            else:
                if streak > limit:
                    end = group.loc[i - 1, 'date']
                    records.append((sid, start, end, streak))
                streak = 0
        if streak > limit:
            end = group.iloc[-1]['date']
            records.append((sid, start, end, streak))
    return records

absent_info = find_absent_streaks(attendance)
absent_df = pd.DataFrame(absent_info, columns=[
    'student_id', 'from_date', 'to_date', 'days_absent'
])

data = pd.merge(absent_df, students, on='student_id')

def valid(email):
    return re.match(r'^[a-zA-Z_][a-zA-Z0-9_]*@[a-zA-Z]+\.(com)$', email) is not None

data['email'] = data['parent_email'].apply(lambda x: x if valid(x) else None)

def message(row):
    if row['email']:
        return f"Dear Parent, your child {row['student_name']} was absent from {row['from_date'].date()} to {row['to_date'].date()} for {row['days_absent']} days. Please ensure their attendance improves."
    return None

data['msg'] = data.apply(message, axis=1)

final = data[['student_id', 'from_date', 'to_date', 'days_absent', 'email', 'msg']]
print(final)


   student_id  from_date    to_date  days_absent                     email  \
0         101 2024-03-02 2024-03-05            4  alice_parent@example.com   
1         102 2024-03-02 2024-03-05            4    bob_parent@example.com   
2         103 2024-03-02 2024-03-05            4                      None   
3         104 2024-03-02 2024-03-05            4                      None   
4         105 2024-03-02 2024-03-05            4     eva_white@example.com   

                                                 msg  
0  Dear Parent, your child Alice Johnson was abse...  
1  Dear Parent, your child Bob Smith was absent f...  
2                                               None  
3                                               None  
4  Dear Parent, your child Eva White was absent f...  
